## Namespaces for pipeline grouping

Namespaces allow you to group nodes, ensuring clear dependencies and separation within a pipeline while maintaining a consistent structure. Like with pipelines or tags, you can enable selective execution using namespaces, and you cannot run more than one namespace simultaneously — Kedro allows executing one namespace at a time.

Defining namespace at Pipeline-level: When applying a namespace at the pipeline level, Kedro automatically renames all inputs, outputs, and parameters within that pipeline. You will need to update your catalog accordingly. If you don’t want to change the names of your inputs, outputs, or parameters with the `namespace_name`. prefix while using a namespace, you should list these objects inside the corresponding parameters of the `pipeline()` creation function. For example:

```python
return pipeline(
    base_pipeline,
    namespace = "new_namespaced_pipeline", # With that namespace, "new_namespaced_pipeline" prefix will be added to inputs, outputs, params, and node names
    inputs={"the_original_input_name"}, # Inputs remain the same, without namespace prefix
)
```

Namespaces allow you to group your nodes and pipelines more efficiently in deployment, for example, when running your pipeline as a Databricks Job or Asset Bundle. 

To add a namespace to a Kedro pipeline, you can use the `namespace` argument in the `pipeline()` function. This argument accepts a string that will be used as a prefix for all nodes, inputs, outputs, and parameters within that pipeline. Note that you'll have to update your catalog accordingly, as Kedro expects that all inputs, outputs, and parameters within that pipeline include the namespace prefix. 

For example, to add a namespace to our `data_science` pipeline you'll have to modifify the code to:

In [ ]:
# data_science/pipeline.py


def create_pipeline(**kwargs) -> Pipeline:
    return pipeline(
        [
            node(
                func=split_data,
                inputs=["model_input_table", "params:model_options"],
                outputs=["X_train", "X_test", "y_train", "y_test"],
                name="split_data_node",
            ),
            node(
                func=train_model,
                inputs=["X_train", "y_train"],
                outputs="regressor",
                name="train_model_node",
            ),
            node(
                func=evaluate_model,
                inputs=["regressor", "X_test", "y_test"],
                outputs=None,
                name="evaluate_model_node",
            ),
        ],
        namespace="ds",
    )

### Update catalog.yml to add namespace prefixes to relevant datasets

```yaml
ds.model_input_table:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: model_input_table
  write_mode: overwrite
```

### Update parameters.yml to add namespace prefixes to relevant parameters

```yaml
ds.model_options:
  test_size: 0.2
  random_state: 3
  features:
    - engines
    - passenger_capacity
    - crew
    - d_check_complete
    - moon_clearance_complete
    - iata_approved
    - company_rating
    - review_scores_rating
```